# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [1]:
#Libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
spaceship = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
spaceship.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


Now perform the same as before:
- Feature Scaling
- Feature Selection


In [5]:
#your code here
print(spaceship.shape)
print(spaceship.dtypes)
print(spaceship.isnull())

(8693, 14)
PassengerId      object
HomePlanet       object
CryoSleep        object
Cabin            object
Destination      object
Age             float64
VIP              object
RoomService     float64
FoodCourt       float64
ShoppingMall    float64
Spa             float64
VRDeck          float64
Name             object
Transported        bool
dtype: object
      PassengerId  HomePlanet  CryoSleep  Cabin  Destination    Age    VIP  \
0           False       False      False  False        False  False  False   
1           False       False      False  False        False  False  False   
2           False       False      False  False        False  False  False   
3           False       False      False  False        False  False  False   
4           False       False      False  False        False  False  False   
...           ...         ...        ...    ...          ...    ...    ...   
8688        False       False      False  False        False  False  False   
8689        Fal

In [6]:
df= spaceship.dropna()

In [7]:
df.drop(['PassengerId', 'Name'], axis=1, inplace=True)

<ipython-input-7-3a77b90f4137>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['PassengerId', 'Name'], axis=1, inplace=True)


In [8]:
# Creating dummy variables
df = pd.get_dummies(df, drop_first=True)

In [9]:
from sklearn.model_selection import train_test_split

# Define features and target
X = df.drop('Transported', axis=1)
y = df['Transported']

# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [10]:
#your code here
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the KNN model
knn = KNeighborsClassifier(n_neighbors=5)

# Fit the model to the training data
knn.fit(X_train, y_train)

# Predict on the test set
y_pred = knn.predict(X_test)



- Evaluate your model

In [11]:
#your code here
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print("Classification Report:")
print(report)

Accuracy: 0.7760968229954615
Classification Report:
              precision    recall  f1-score   support

       False       0.77      0.77      0.77       653
        True       0.78      0.78      0.78       669

    accuracy                           0.78      1322
   macro avg       0.78      0.78      0.78      1322
weighted avg       0.78      0.78      0.78      1322



**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [12]:
#your code here
from sklearn.model_selection import GridSearchCV

# Define parameter grid
param_grid = {
    'n_neighbors': [3, 5, 7, 9],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski']
}




/usr/local/lib/python3.10/dist-packages/numpy/ma/core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best parameters found:  {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}
Accuracy with best params: 0.7859304084720121
              precision    recall  f1-score   support

       False       0.80      0.76      0.78       653
        True       0.78      0.81      0.79       669

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322



- Run Grid Search

In [13]:
# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid, cv=5, scoring='accuracy', n_jobs=-1)

# Fit to the data
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Update model with best parameters
knn_best = KNeighborsClassifier(**best_params)
knn_best.fit(X_train, y_train)


Best parameters found:  {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'uniform'}


KNeighborsClassifier(metric='manhattan', n_neighbors=9)

- Evaluate your model

In [14]:

# Predict and evaluate
y_pred_best = knn_best.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
report_best = classification_report(y_test, y_pred_best)
print(f"Accuracy with best params: {accuracy_best}")
print(report_best)

Accuracy with best params: 0.7859304084720121
              precision    recall  f1-score   support

       False       0.80      0.76      0.78       653
        True       0.78      0.81      0.79       669

    accuracy                           0.79      1322
   macro avg       0.79      0.79      0.79      1322
weighted avg       0.79      0.79      0.79      1322

